# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import base64
import time
import math
import datetime
import os
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm
from pathlib import Path
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
from joblib import Parallel , delayed
import multiprocessing as mp 

import torch
import torchvision
import torch.distributed as dist
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from collections import defaultdict, deque

# 데이터 로드

In [2]:
base_dir = '/home/jovyan/nas/1_user/mhyeom@agilesoda.ai/235855_병변 검출 AI 경진대회_data'

In [3]:
train_files = sorted(glob(f'{base_dir}/train/*'))
test_files = sorted(glob(f'{base_dir}/test/*'))

In [4]:
label_info = pd.read_csv((f"{base_dir}/class_id_info.csv"))
categories = {i[0]:i[1]-1 for i in label_info.to_numpy()}
label_info

,class,class_id
0,01_ulcer,1
1,02_mass,2
2,04_lymph,3
3,05_bleeding,4


In [17]:
categories

{'01_ulcer': 0, '02_mass': 1, '04_lymph': 2, '05_bleeding': 3}

In [2]:
def xyxy2coco(xyxy):
    x1,y1,x2,y2 =xyxy
    w,h =  x2-x1, y2-y1
    return [x1,y1,w,h] 

def xyxy2yolo(xyxy):
    
    x1,y1,x2,y2 =xyxy
    w,h =  x2-x1, y2-y1
    xc = x1 + int(np.round(w/2)) # xmin + width/2
    yc = y1 + int(np.round(h/2)) # ymin + height/2
    return [xc/IMG_SIZE,yc/IMG_SIZE,w/IMG_SIZE,h/IMG_SIZE] 

def scale_bbox(img, xyxy):
    # Get scaling factor
    scale_x = IMG_SIZE/img.shape[1]
    scale_y = IMG_SIZE/img.shape[0]
    
    x1,y1,x2,y2 =xyxy
    x1 = int(np.round(x1*scale_x, 4))
    y1 = int(np.round(y1*scale_y, 4))
    x2 = int(np.round(x2*scale_x, 4))
    y2= int(np.round(y2*scale_y, 4))

    return [x1, y1, x2, y2] # xmin, ymin, xmax, ymax

def save_image_label(json_file,mode): 
    with open(json_file,'r') as f: 
        json_file =json.load(f)

    image_id = json_file['file_name'].replace('.json','')
    
    # decode image data
    image = np.frombuffer(base64.b64decode(json_file['imageData']), np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    cv2.imwrite(str(new_image_path / (image_id + '.png')) ,image)
    
    # extract bbox
    origin_bbox = []
    if mode == 'train':
        with open(new_label_path / (image_id + '.txt'), 'w') as f:
            for i in json_file['shapes']: 
                bbox = i['points'][0] + i['points'][2]
                origin_bbox.append(bbox)
                bbox = scale_bbox(image,bbox)
                bbox = xyxy2yolo(bbox)
                
                labels = [categories[i['label']]]+bbox
                f.writelines([f'{i} ' for i in labels] + ['\n']) 
    return origin_bbox

In [19]:
# 저장할 파일 경로
IMG_SIZE = 576
save_path = Path('./train_data')
new_image_path = save_path / 'images' # image폴더 
new_label_path = save_path / 'labels' # label폴더

new_image_path.mkdir(parents=True,exist_ok=True)
new_label_path.mkdir(parents=True,exist_ok=True)

# data를 생성하기 위해 mlutiprocessing 적용
base_path = Path(base_dir)
train_path_list = list((base_path /'train').glob('train*'))
tmp = Parallel(n_jobs=mp.cpu_count(),prefer="threads")(delayed(save_image_label)(str(train_json),'train') for train_json in tqdm(train_path_list))

  0%|          | 0/62622 [00:00<?, ?it/s]

In [20]:
images_path = list(new_image_path.glob('*'))

train_path_list,valid_path_list = train_test_split(images_path,test_size=0.15,random_state=42)

In [21]:
with open('train_dataset.txt', 'w') as f:
    f.writelines([f'{i}\n' for i in train_path_list])
with open('valid_dataset.txt', 'w') as f:
    f.writelines([f'{i}\n' for i in valid_path_list]) 

In [22]:
import yaml

data_yaml = dict(
    train = './train_dataset.txt',
    val = './valid_dataset.txt',
    nc = 4,
    names = ['01_ulcer','02_mass','04_lymph','05_bleeding']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('endoscopy.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [ ]:
# single gpu
!python  ./yolov5/train.py \
         --img 128 \
         --batch 64\
         --epochs 100\
         --data ./endoscopy.yaml\
         --cfg yolov5s.yaml\
         --weights ../input/ultralyticsyolov5aweights/yolov5s.pt\
         --project yolov5-endoscopy\
         --save-period 1\
         --name endoscopy_s_128_100\
         --device 2

train: weights=../input/ultralyticsyolov5aweights/yolov5s.pt, cfg=yolov5s.yaml, data=./endoscopy.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=64, imgsz=128, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=2, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5-endoscopy, name=endoscopy_s_128_100, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-100-g5185981 torch 1.7.1+cu110 CUDA:2 (Graphics Device, 24256MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=

In [31]:
# multi gpu
!python  -m torch.distributed.launch --nproc_per_node 2 ./yolov5/train.py \
         --img 576 \
         --batch 64\
         --epochs 40\
         --data ./endoscopy.yaml\
         --cfg yolov5l.yaml\
         --weights ../input/ultralyticsyolov5aweights/yolov5l.pt\
         --project yolov5-endoscopy\
         --save-period 1\
         --name endoscopy_l_576\
         --device 2,3

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
train: weights=../input/ultralyticsyolov5aweights/yolov5l.pt, cfg=yolov5l.yaml, data=./endoscopy.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=40, batch_size=64, imgsz=576, rect=False, resume=False, nosave=False, noval=False, noau

In [ ]:
# 저장할 파일 경로
IMG_SIZE = 576
save_path = Path('./test_data')
new_image_path = save_path / 'images' # image폴더 
new_label_path = save_path / 'labels' # label폴더

new_image_path.mkdir(parents=True,exist_ok=True)
new_label_path.mkdir(parents=True,exist_ok=True)

# data를 생성하기 위해 mlutiprocessing 적용
base_path = Path(base_dir)
test_path_list = list((base_path /'test').glob('test*'))
tmp = Parallel(n_jobs=mp.cpu_count(),prefer="threads")(delayed(save_image_label)(str(test_json),'test') for test_json in tqdm(test_path_list))

In [6]:
base_path = Path(base_dir)
test_path_list = list((base_path /'test').glob('test*'))

In [ ]:
!python ./yolov5/detect.py \
        --weights ./yolov5-endoscopy/endoscopy_l_576/weights/best.pt \
        --img 576 \
        --conf 0.4 \
        --source ./test_data/images/\
        --device 2\
        --save-txt\
        --save-conf

detect: weights=['./yolov5-endoscopy/endoscopy_l_576/weights/best.pt'], source=./test_data/images/, imgsz=[576, 576], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=2, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-100-g5185981 torch 1.7.1+cu110 CUDA:2 (Graphics Device, 24256MiB)

Fusing layers... 
Model Summary: 367 layers, 46124433 parameters, 0 gradients, 107.8 GFLOPs
image 1/20874 /home/jovyan/nas/1_user/jspark@agilesoda.ai/D1/test_data/images/test_200000.png: 576x576 Done. (0.026s)
image 2/20874 /home/jovyan/nas/1_user/jspark@agilesoda.ai/D1/test_data/images/test_200001.png: 576x576 2 01_ulcers, Done. (0.024s)
image 3/20874 /home/jovyan/nas/1_user/jspark@agilesoda.ai/D1/test_data/images/test_200002.png: 576x576 1 02_mass, Do

In [10]:
test_lab_list = os.listdir('./yolov5/runs/detect/exp2/labels/')

In [11]:
test_lab_list.sort()

In [20]:
submission_pd = pd.DataFrame(columns = ['file_name','class_id','confidence','point1_x','point1_y','point2_x','point2_y','point3_x','point3_y','point4_x','point4_y'])

In [13]:
def yolobbox2bbox(xywh):
    x,y,w,h = xywh
    x1, y1 = x-w/2, y-h/2
    x2, y2 = x+w/2, y+h/2
    return x1, y1, x2, y2

- 박스 여러개 추가

In [21]:
for test_lists in tqdm(test_lab_list):
    f = open(f'./yolov5/runs/detect/exp2/labels/{test_lists}', "r")
    lines = f.read().split('\n')
    for boxes in lines:
        if boxes == '':
            continue
        annotation = boxes.split(' ')
        box_coordinates = [np.float32(x) for x in boxes.split(' ')[1:5]]
        box_coordinates = [x*576 for x in yolobbox2bbox(box_coordinates)]
        xs = [box_coordinates[0], box_coordinates[2]]
        ys = [box_coordinates[1], box_coordinates[3]]
        sub_file_name = test_lists.split('.')[0] + '.json'
        sub_class_id = int(annotation[0]) + 1
        sub_confidence = np.float32(annotation[-1])
        sub_point1_x = min(xs)
        sub_point1_y = min(ys)
        sub_point2_x = max(xs)
        sub_point2_y = min(ys)
        sub_point3_x = max(xs)
        sub_point3_y = max(ys)
        sub_point4_x = min(xs)
        sub_point4_y = max(ys)
        sub_list = [sub_file_name, 
                    sub_class_id, 
                    sub_confidence,
                    sub_point1_x,
                    sub_point1_y,
                    sub_point2_x,
                    sub_point2_y,
                    sub_point3_x,
                    sub_point3_y,
                    sub_point4_x,
                    sub_point4_y]
        sub_pd = pd.Series(sub_list, index = submission_pd.columns)
        submission_pd = submission_pd.append(sub_pd, ignore_index=True)
    f.close()

  0%|          | 0/18349 [00:00<?, ?it/s]

In [22]:
submission_pd

,file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
0,test_200001.json,1,0.673867,124.000124,332.000364,265.000315,332.000364,265.000315,471.000109,124.000124,471.000109
1,test_200001.json,1,0.679013,408.000092,199.000224,525.000092,199.000224,525.000092,315.000288,408.000092,315.000288
2,test_200002.json,2,0.800770,222.999845,347.000247,341.999717,347.000247,341.999717,475.000119,222.999845,475.000119
3,test_200003.json,2,0.683024,244.800282,10.799698,541.800282,10.799698,541.800282,347.399712,244.800282,347.399712
4,test_200003.json,2,0.789123,414.000000,243.000275,576.000000,243.000275,576.000000,468.000275,414.000000,468.000275
...,...,...,...,...,...,...,...,...,...,...,...
23457,test_220870.json,3,0.793532,331.199987,19.799706,381.599986,19.799706,381.599986,169.199710,331.199987,169.199710
23458,test_220871.json,4,0.614160,366.999840,153.999945,432.999649,153.999945,432.999649,222.000200,366.999840,222.000200
23459,test_220872.json,1,0.716499,366.999836,374.999896,447.999836,374.999896,447.999836,472.999959,366.999836,472.999959
23460,test_220873.json,1,0.564129,237.999753,162.999928,343.999881,162.999928,343.999881,252.999928,237.999753,252.999928


In [144]:
max([box_coordinates[0], box_coordinates[2]])

455.0002727508545

In [23]:
submission_pd.to_csv('./Results/yolov5l_576_eph10.csv', encoding = 'utf-8', index = False)

In [24]:
submission_pd['class_id'].value_counts()

1    10843
4     5554
3     3716
2     3349
Name: class_id, dtype: int64

In [122]:
temp['class_id'].value_counts()

1    7785
4    3394
2    3015
3    2131
Name: class_id, dtype: int64

In [5]:
os.listdir('/home/jovyan/nas/1_user/jspark@agilesoda.ai/D1/yolov5/runs/detect/exp3/')

['test_212299.png',
 'test_211032.png',
 'test_213072.png',
 'test_210117.png',
 'test_209402.png',
 'test_211811.png',
 'test_204214.png',
 'test_218637.png',
 'test_211299.png',
 'test_210229.png',
 'test_212385.png',
 'test_213627.png',
 'test_212559.png',
 'test_212787.png',
 'test_200470.png',
 'test_215407.png',
 'test_207765.png',
 'test_204272.png',
 'test_203926.png',
 'test_212484.png',
 'test_207191.png',
 'test_217981.png',
 'test_218944.png',
 'test_219509.png',
 'test_208576.png',
 'test_207874.png',
 'test_216010.png',
 'test_217765.png',
 'test_216908.png',
 'test_208510.png',
 'test_219882.png',
 'test_201490.png',
 'test_212432.png',
 'test_204137.png',
 'test_207689.png',
 'test_206306.png',
 'test_211338.png',
 'test_202458.png',
 'test_218231.png',
 'test_220221.png',
 'test_204345.png',
 'test_213788.png',
 'test_206910.png',
 'test_209195.png',
 'test_200912.png',
 'test_206007.png',
 'test_203124.png',
 'test_202683.png',
 'test_209507.png',
 'test_204870.png',


In [6]:
cv2.imread(str('/home/jovyan/nas/1_user/jspark@agilesoda.ai/D1/yolov5/runs/detect/exp3/test_212299.png'))[:,:,::-1].shape

(576, 576, 3)